## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos para clustering

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario considerando su uso para los métodos de clusterización.

---

In [2]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

Se parte de los mismos datasets con los que se realizó la integración anterior.

In [4]:
df_creditos = pd.read_csv("../../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [5]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = []
col_eliminar_tarjetas = ['nivel_tarjeta']

# Se ejecuta la operación

df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [6]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,24.0,2.0


Limpieza de los datos (filtrado a nivel de filas)

In [7]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,ok
1,768805383.0,21,1000,2,5.0,PROPIA,9600,EDUCACIÓN,0.10,11.14,0,N,ok
2,818770008.0,25,5500,3,1.0,HIPOTECA,9600,SALUD,0.57,12.87,1,N,err
3,713982108.0,23,35000,2,4.0,ALQUILER,65500,SALUD,0.53,15.23,1,N,ok
4,710821833.0,24,35000,4,8.0,ALQUILER,54400,SALUD,0.55,14.27,1,Y,ok


In [8]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

temp = df_creditos[df_creditos['edad'] < 90]

# Otro filtro posible: por la regla de negocio agregada

temp_c = temp[temp['regla_pct_ingresos'] == 'ok']

print(f"Filas después del filtro: {temp_c.shape[0]}")

Filas antes del filtro: 10127
Filas después del filtro: 10108


Integración de datos

In [9]:
df_integrado = pd.merge(temp_c, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrado con los filtros realizados: {coincidencias}")

Filas del dataset integrado con los filtros realizados: 10108


In [10]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 22


#### Transformación de atributos

Observación general del dataset

In [9]:
df_integrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10108 entries, 0 to 10107
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_cliente           10108 non-null  float64
 1   edad                 10108 non-null  int64  
 2   importe_solicitado   10108 non-null  int64  
 3   duracion_credito     10108 non-null  int64  
 4   antiguedad_empleado  9772 non-null   float64
 5   situacion_vivienda   10108 non-null  object 
 6   ingresos             10108 non-null  int64  
 7   objetivo_credito     10108 non-null  object 
 8   pct_ingreso          10108 non-null  float64
 9   tasa_interes         9198 non-null   float64
 10  estado_credito       10108 non-null  int64  
 11  falta_pago           10108 non-null  object 
 12  regla_pct_ingresos   10108 non-null  object 
 13  antiguedad_cliente   10108 non-null  float64
 14  estado_civil         10108 non-null  object 
 15  estado_cliente       10108 non-null 

##### Procesamiento de valores nulos

En este caso se van a filtrar (eliminar)

In [12]:
df_filtrado = df_integrado[(df_integrado['tasa_interes'].notnull()) & (df_integrado['antiguedad_empleado'].notnull())]
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8885 entries, 0 to 10107
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_cliente           8885 non-null   float64
 1   edad                 8885 non-null   int64  
 2   importe_solicitado   8885 non-null   int64  
 3   duracion_credito     8885 non-null   int64  
 4   antiguedad_empleado  8885 non-null   float64
 5   situacion_vivienda   8885 non-null   object 
 6   ingresos             8885 non-null   int64  
 7   objetivo_credito     8885 non-null   object 
 8   pct_ingreso          8885 non-null   float64
 9   tasa_interes         8885 non-null   float64
 10  estado_credito       8885 non-null   int64  
 11  falta_pago           8885 non-null   object 
 12  regla_pct_ingresos   8885 non-null   object 
 13  antiguedad_cliente   8885 non-null   float64
 14  estado_civil         8885 non-null   object 
 15  estado_cliente       8885 non-null   objec

##### Procesamiento de atributos nominales

Se realiza una binarización por aquellos métodos que no pueden operar con atributos no numéricos.

In [13]:
data = pd.get_dummies(df_filtrado)
data.head()

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,ingresos,pct_ingreso,tasa_interes,estado_credito,antiguedad_cliente,...,estado_cliente_ACTIVO,estado_cliente_PASIVO,genero_F,genero_M,nivel_educativo_DESCONOCIDO,nivel_educativo_POSGRADO_COMPLETO,nivel_educativo_POSGRADO_INCOMPLETO,nivel_educativo_SECUNDARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_INCOMPLETO
0,713061558.0,22,35000,3,123.0,59000,0.59,16.02,1,36.0,...,True,False,False,True,False,False,False,False,True,False
1,768805383.0,21,1000,2,5.0,9600,0.10,11.14,0,39.0,...,True,False,False,True,False,False,False,True,False,False
2,713982108.0,23,35000,2,4.0,65500,0.53,15.23,1,36.0,...,True,False,False,True,False,False,False,False,True,False
3,710821833.0,24,35000,4,8.0,54400,0.55,14.27,1,54.0,...,True,False,False,True,True,False,False,False,False,False
4,769911858.0,21,2500,2,2.0,9900,0.25,7.14,1,34.0,...,True,False,True,False,False,False,False,True,False,False


Se exportan los resultados para poder utilizarlos en las operaciones de clustering

In [14]:
data.to_csv("../../../data/final/datos_clusterizacion.csv", sep=';', index=False)

In [15]:
print(f"Cantidad de filas del dataset integrado: {data.shape[0]}")
print(f"Cantidad de columnas del dataset integrado: {data.shape[1]}")

Cantidad de filas del dataset integrado: 8885
Cantidad de columnas del dataset integrado: 41
